In [1]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 69kB/s 
     |████████████████████████████████| 204kB 17.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=afcbcfa3fbaaa2d9540cc59b1e5c3f7ecb9369afcd9818cd951996b5a5e91ed9
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [4]:
spark = SparkSession.builder.appName('sparkify-train').getOrCreate()

In [22]:
df = spark.read.parquet('/content/drive/MyDrive/datasets/dsnd-sparkify/ml_df.parquet')
df = df.withColumn('label', F.when(F.col("is_churn"), 1).otherwise(0))
df.show(5)

+-------+-------------------+--------+----------+-------------+------------------+----------+-----------+---------+------------------+----------+-----+------+-----+-----+-------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----------+-----+-----+-----+-----+-----+-----+-----+-----+--------+--------+-----+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----------+-----+-----+-----+-----+-----+-----+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+--------+-----+-----+-----+-----+-----+
| userId|              up_ts|is_churn|song_count|subs_duration|         song_rate|n_playlist|thumbs_down|thumbs_up|      avg_sess_len|sess_count| ipad|iphone|linux|macos|windows|   AK|   A

In [56]:
train_df, test_df = df.randomSplit([0.7, 0.3], seed=9)

In [15]:
df.filter(df.is_churn == True).agg(F.mean(df.subs_duration), F.stddev(df.subs_duration)).show()

+------------------+--------------------------+
|avg(subs_duration)|stddev_samp(subs_duration)|
+------------------+--------------------------+
|14.668282828282829|        13.809922024885187|
+------------------+--------------------------+



In [57]:
featureCols = ["song_count", "subs_duration", "song_rate", "n_playlist", "thumbs_down",
               "thumbs_up", "avg_sess_len", "sess_count", "ipad", "iphone", "linux",
               "macos", "windows"]
# featureCols = df.columns[3:]

assembler = VectorAssembler(inputCols=featureCols, outputCol="features")

train_df = assembler.transform(train_df)
test_df = assembler.transform(test_df)

In [51]:
model = GBTClassifier(maxDepth=4)
model1 = model.fit(train_df)

In [58]:
preds_df = model1.transform(test_df)

In [61]:
binary_evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
res = binary_evaluator.evaluate(preds_df)
res

Py4JJavaError: ignored